# Packages Installs, imports, and presets

In [24]:
from scapy.all import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from collections import defaultdict
import os

In [25]:
INPUT_SIZE = (256,256,3)

In [26]:
df = pd.read_parquet('data/iec104_15.parquet')

## Removing missing features

In [27]:
df.shape

(518377, 1347)

In [28]:
df['tv_sec']=df['tv_sec'].astype(int)
df['tv_usec']=df['tv_usec'].astype(int)

# Model training

## Dataset preparation

In [29]:
df[['label']].value_counts()

label        
normal           400317
c_rd_na_1_DoS     20167
c_rp_na_1_DoS     19949
c_sc_na_1_DoS     16615
c_rp_na_1         11517
c_se_na_1_DoS     10854
c_rd_na_1         10821
c_ci_na_1_DoS      9146
c_se_na_1          6960
c_sc_na_1          6767
c_ci_na_1          5264
Name: count, dtype: int64

In [30]:
df[['label']]

,label
flow,
1_192.168.1.13_192.168.1.20_2404_49259_TCP_c_rp_na_1_normal,normal
2_192.168.1.13_192.168.1.20_2404_49259_TCP_c_rp_na_1_normal,normal
2_192.168.1.13_192.168.1.20_2404_49259_TCP_c_rp_na_1_normal,normal
2_192.168.1.13_192.168.1.20_2404_49259_TCP_c_rp_na_1_normal,normal
2_192.168.1.13_192.168.1.20_2404_49259_TCP_c_rp_na_1_normal,normal
...,...
1_192.168.1.29_192.168.1.22_46861_2404_TCP_c_sc_na_1_DoS_c_sc_na_1_DoS,c_sc_na_1_DoS
1_192.168.1.29_192.168.1.22_46861_2404_TCP_c_sc_na_1_DoS_c_sc_na_1_DoS,c_sc_na_1_DoS
1_192.168.1.29_192.168.1.22_46871_2404_TCP_c_sc_na_1_c_sc_na_1,c_sc_na_1


In [31]:
from feature_engine.encoding import OrdinalEncoder

In [32]:
od = OrdinalEncoder(encoding_method='arbitrary')
od.fit(df[['label']])
df[['label']]= od.transform(df[['label']])

In [33]:
TRAIN_RATE= 0.8
import random
indexes = list(set(df.index))
random.Random(1).shuffle(indexes)
TRAIN_SIZE = int(len(indexes)*TRAIN_RATE)
TEST_SIZE = len(indexes)-TRAIN_SIZE
TRAIN_SAMPLES = indexes[:TRAIN_SIZE]
TEST_SAMPLES = indexes[:TEST_SIZE]
df_train_initial = df.loc[TRAIN_SAMPLES]
df_test = df.loc[TEST_SAMPLES]
# df_test = df.iloc[TRAIN_SAMPLES:]
# del df

In [34]:
indexes = list(set(df_train_initial.index))
random.Random(1).shuffle(indexes)
TRAIN_SIZE = int(len(indexes)*TRAIN_RATE)
TEST_SIZE = len(indexes)-TRAIN_SIZE
TRAIN_SAMPLES = indexes[:TRAIN_SIZE]
TEST_SAMPLES = indexes[:TEST_SIZE]
df_train = df_train_initial.loc[TRAIN_SAMPLES]
df_validation = df_train_initial.loc[TEST_SAMPLES]

In [35]:
df_train.shape, df_validation.shape, df_test.shape

((369037, 1347), (69233, 1347), (82498, 1347))

In [36]:
indexes[:3]

['198_192.168.1.20_192.168.1.21_49723_2404_TCP_c_sc_na_1_normal',
 '2_192.168.1.29_192.168.1.19_40327_2404_TCP_c_rd_na_1_c_rd_na_1',
 '106_192.168.1.20_192.168.1.25_49725_2404_TCP_c_sc_na_1_normal']

In [37]:
len(set(df_train.index))

63161

In [39]:
display(df_train['label'].value_counts())
display(df_validation['label'].value_counts())
display(df_test['label'].value_counts())

label
0     293571
4      12783
6      12705
10     10805
2       7394
1       7031
3       6953
8       5681
9       4510
7       4345
5       3259
Name: count, dtype: int64

label
0     50337
4      3229
6      3184
10     2772
3      1830
2      1790
1      1708
8      1400
7      1067
9      1051
5       865
Name: count, dtype: int64

label
0     58855
4      3987
6      3969
10     3361
3      2316
2      2281
1      2184
8      1820
7      1342
9      1321
5      1062
Name: count, dtype: int64

In [ ]:
del df_train_initial
del df

In [ ]:
assert df_train.groupby(df_train.index)['label'].nunique().max()==1

In [ ]:
len(set(df_train.index))

63161

In [ ]:
df_train

,tv_sec,tv_usec,ipv4_ver_0,ipv4_ver_1,ipv4_ver_2,ipv4_ver_3,ipv4_hl_0,ipv4_hl_1,ipv4_hl_2,ipv4_hl_3,...,payload_bit_375,payload_bit_376,payload_bit_377,payload_bit_378,payload_bit_379,payload_bit_380,payload_bit_381,payload_bit_382,payload_bit_383,label
flow,,,,,,,,,,,,,,,,,,,,,
198_192.168.1.20_192.168.1.21_49723_2404_TCP_c_sc_na_1_normal,1588104246,854923,0,1,0,0,0,1,0,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
198_192.168.1.20_192.168.1.21_49723_2404_TCP_c_sc_na_1_normal,1588104249,735865,0,1,0,0,0,1,0,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
198_192.168.1.20_192.168.1.21_49723_2404_TCP_c_sc_na_1_normal,1588104255,737112,0,1,0,0,0,1,0,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
2_192.168.1.29_192.168.1.19_40327_2404_TCP_c_rd_na_1_c_rd_na_1,1591361814,729264,0,1,0,0,0,1,0,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,3
2_192.168.1.29_192.168.1.19_40327_2404_TCP_c_rd_na_1_c_rd_na_1,1591361824,742618,0,1,0,0,0,1,0,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24_192.168.1.26_192.168.1.20_2404_49258_TCP_c_rp_na_1_normal,1591431620,978610,0,1,0,0,0,1,0,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
24_192.168.1.26_192.168.1.20_2404_49258_TCP_c_rp_na_1_normal,1591431622,621116,0,1,0,0,0,1,0,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
24_192.168.1.26_192.168.1.20_2404_49258_TCP_c_rp_na_1_normal,1591431623,558636,0,1,0,0,0,1,0,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0


In [ ]:
import numpy as np
from scapy.all import IP, send
def create_tcp_packet_from_binary(binary_sequence):
    """
    Creates a TCP packet from a given binary sequence with -1 indicating "don't care" bits.
    
    Args:
        binary_sequence (numpy.ndarray): The binary sequence with 0, 1, and -1 values.
    
    Returns:
        TCP: The constructed TCP packet.
    """
    # Ensure the binary_sequence is a NumPy array
    binary_sequence = np.array(binary_sequence)

    # Replace -1 with 0 (default value) in the binary sequence
    cleaned_sequence = np.where(binary_sequence == -1, 0, binary_sequence)

    # Convert the cleaned binary sequence to a binary string
    binary_string = ''.join(map(str, cleaned_sequence))

    # Group binary string into bytes (8 bits each)
    bytes_sequence = [binary_string[i:i+8] for i in range(0, len(binary_string), 8)]

    # Convert each byte from binary string to integer
    byte_values = [int(byte, 2) for byte in bytes_sequence]

    # Construct the TCP packet
    tcp_packet = TCP()

    # Set the fields of the TCP packet based on the byte values
    tcp_packet.sport = (byte_values[0] << 8) | byte_values[1]  # Source port
    tcp_packet.dport = (byte_values[2] << 8) | byte_values[3]  # Destination port
    tcp_packet.seq = (byte_values[4] << 24) | (byte_values[5] << 16) | (byte_values[6] << 8) | byte_values[7]  # Sequence number
    tcp_packet.ack = (byte_values[8] << 24) | (byte_values[9] << 16) | (byte_values[10] << 8) | byte_values[11]  # Acknowledgment number
    tcp_packet.dataofs = byte_values[12] >> 4  # Data offset (4 bits)
    tcp_packet.reserved = (byte_values[12] & 0x0F)  # Reserved (4 bits)
    tcp_packet.flags = byte_values[13]  # Flags
    tcp_packet.window = (byte_values[14] << 8) | byte_values[15]  # Window size
    tcp_packet.chksum = (byte_values[16] << 8) | byte_values[17]  # Checksum
    tcp_packet.urgptr = (byte_values[18] << 8) | byte_values[19]  # Urgent pointer

    return tcp_packet

def create_ipv4_packet_from_binary(binary_sequence):
    """
    Creates an IPv4 packet from a given binary sequence with -1 indicating "don't care" bits.
    
    Args:
        binary_sequence (numpy.ndarray): The binary sequence with 0, 1, and -1 values.
    
    Returns:
        IP: The constructed IPv4 packet.
    """
    # Ensure the binary_sequence is a NumPy array
    binary_sequence = np.array(binary_sequence)

    # Replace -1 with 0 (default value) in the binary sequence
    cleaned_sequence = np.where(binary_sequence == -1, 0, binary_sequence)

    # Convert the cleaned binary sequence to a binary string
    binary_string = ''.join(map(str, cleaned_sequence))

    # Group binary string into bytes (8 bits each)
    bytes_sequence = [binary_string[i:i+8] for i in range(0, len(binary_string), 8)]

    # Convert each byte from binary string to integer
    byte_values = [int(byte, 2) for byte in bytes_sequence]

    # Construct the IPv4 packet
    ip_packet = IP()

    # Set the fields of the IP packet based on the byte values
    ip_packet.version = byte_values[0] >> 4
    ip_packet.ihl = byte_values[0] & 0x0F
    ip_packet.tos = byte_values[1]
    ip_packet.len = (byte_values[2] << 8) | byte_values[3]
    ip_packet.id = (byte_values[4] << 8) | byte_values[5]
    ip_packet.flags = byte_values[6] >> 5
    ip_packet.frag = ((byte_values[6] & 0x1F) << 8) | byte_values[7]
    ip_packet.ttl = byte_values[8]
    ip_packet.proto = byte_values[9]
    ip_packet.chksum = (byte_values[10] << 8) | byte_values[11]
    ip_packet.src = '.'.join(map(str, byte_values[12:16]))
    ip_packet.dst = '.'.join(map(str, byte_values[16:20]))

    return ip_packet


In [ ]:
import importlib
import flowmeter #import the module here, so that it can be reloaded.
importlib.reload(flowmeter)

<module 'flowmeter' from '/home/hwerneck/fast-ids/flowmeter.py'>

In [ ]:
def create_payload_from_binary(binary_sequence):
    """
    Creates a payload from a given binary sequence with -1 indicating "don't care" bits.
    
    Args:
        binary_sequence (numpy.ndarray): The binary sequence with 0, 1, and -1 values.
    
    Returns:
        bytes: The constructed payload.
    """
    # Ensure the binary_sequence is a NumPy array
    binary_sequence = np.array(binary_sequence)

    # Replace -1 with 0 (default value) in the binary sequence
    cleaned_sequence = np.where(binary_sequence == -1, 0, binary_sequence)

    # Convert the cleaned binary sequence to a binary string
    binary_string = ''.join(map(str, cleaned_sequence))

    # Group binary string into bytes (8 bits each)
    bytes_sequence = [binary_string[i:i+8] for i in range(0, len(binary_string), 8)]

    # Convert each byte from binary string to integer
    byte_values = [int(byte, 2) for byte in bytes_sequence]

    # Convert byte values to bytes
    payload = bytes(byte_values)

    return payload

In [ ]:
from tqdm import tqdm

In [ ]:
exit

In [ ]:
def get_flows_statistics(df_input):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=FutureWarning)
        fm=flowmeter.Flowmeter()
        flows_statistics = []
        fgb = df_input.groupby('flow')
        a=0
        for i in tqdm(fgb):
            a=+1
            column, rows = i
            # print(len(rows.filter(regex='ipv4_*').columns))
            # print(len(rows.filter(regex='payload_*').columns))
            # print(len(rows.filter(regex='tcp_*').columns))
            packets = []
            for index, value in rows.iterrows():
                packet_binary=  value.filter(regex='ipv4_*').values
                ipv4 = create_ipv4_packet_from_binary(packet_binary)
                packet_binary=  value.filter(regex='tcp_*').values
                tcp = create_tcp_packet_from_binary(packet_binary)
                packet_binary=  value.filter(regex='payload_*').values
                payload = create_payload_from_binary(packet_binary)
                packet= ipv4/tcp/raw(payload)
                packets.append(packet)
                # packet.show()
            packet_list = PacketList(packets)
            df=fm.build_feature_from_packet_list(packet_list)
            
            label = rows['label'].iloc[0]
            df['label'] = label
            df['flow'] = column
            flows_statistics.append(df)
            display(df.head())
            if a == 3:
                break
        return pd.concat(flows_statistics)

df_train_statistics=get_flows_statistics(df_train)
df_train_statistics.to_parquet('data/df_train_statistics.parquet')

df_validation_statistics=get_flows_statistics(df_validation)
df_validation_statistics.to_parquet('data/df_validation_statistics.parquet')

df_test_statistics=get_flows_statistics(df_test)
df_test_statistics.to_parquet('data/df_test_statistics.parquet')

  0%|                                                                | 0/63161 [00:00<?, ?it/s]

,flow,src,src_port,dst,dst_port,feduration,total_fpackets,total_bpackets,total_fpktl,total_bpktl,...,avgPacketSize,fAvgSegmentSize,fAvgBytesPerBulk,fAvgPacketsPerBulk,fAvgBulkRate,bAvgSegmentSize,bAvgBytesPerBulk,bAvgPacketsPerBulk,bAvgBulkRate,label
0,100_192.168.1.13_192.168.1.20_2404_49259_TCP_c...,192.168.1.13,2404,192.168.1.20,49259,1,1,0,88,0,...,88.0,48.0,0,0,0,NaN,0,0,0,0


  0%|                                                     | 1/63161 [00:02<45:40:29,  2.60s/it]

,flow,src,src_port,dst,dst_port,feduration,total_fpackets,total_bpackets,total_fpktl,total_bpktl,...,avgPacketSize,fAvgSegmentSize,fAvgBytesPerBulk,fAvgPacketsPerBulk,fAvgBulkRate,bAvgSegmentSize,bAvgBytesPerBulk,bAvgPacketsPerBulk,bAvgBulkRate,label
0,100_192.168.1.13_192.168.1.20_2404_49672_TCP_c...,192.168.1.13,2404,192.168.1.20,49672,0.018878,6,0,528,0,...,88.0,48.0,0,0,0,NaN,0,0,0,0


  0%|                                                     | 2/63161 [00:02<21:24:04,  1.22s/it]

...

,flow,src,src_port,dst,dst_port,feduration,total_fpackets,total_bpackets,total_fpktl,total_bpktl,...,avgPacketSize,fAvgSegmentSize,fAvgBytesPerBulk,fAvgPacketsPerBulk,fAvgBulkRate,bAvgSegmentSize,bAvgBytesPerBulk,bAvgPacketsPerBulk,bAvgBulkRate,label
0,102_192.168.1.13_192.168.1.29_2404_39165_TCP_c...,192.168.1.13,2404,192.168.1.29,39165,1,1,0,88,0,...,88.0,48.0,0,0,0,NaN,0,0,0,8


  0%|▏                                                   | 206/63161 [00:52<4:31:56,  3.86it/s]

,flow,src,src_port,dst,dst_port,feduration,total_fpackets,total_bpackets,total_fpktl,total_bpktl,...,avgPacketSize,fAvgSegmentSize,fAvgBytesPerBulk,fAvgPacketsPerBulk,fAvgBulkRate,bAvgSegmentSize,bAvgBytesPerBulk,bAvgPacketsPerBulk,bAvgBulkRate,label
0,102_192.168.1.19_192.168.1.20_2404_49262_TCP_c...,192.168.1.19,2404,192.168.1.20,49262,0.010884,4,0,352,0,...,88.0,48.0,0,0,0,NaN,0,0,0,0


  0%|▏                                                   | 207/63161 [00:53<4:28:54,  3.90it/s]


KeyboardInterrupt: 